In [12]:
from neo4j import GraphDatabase
import pandas as pd
import warnings
from neo4j import GraphDatabase
from dotenv import load_dotenv
load_dotenv()
import xgboost as xgb
from sklearn.model_selection import train_test_split


# NumPy関連のRuntimeWarningを無効化
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [13]:
# DB接続処理
# Neo4jに接続
uri = "bolt://localhost:7687"
user = "neo4j"
password = "abcd7890"

# ドライバを作成
driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
# レシピのページランクやAromaのページランクを取得する
# FIX THIS CODE
def drop_projection_graph(tx):
    query = """
    CALL gds.graph.drop('MbfRux')
    """
    tx.run(query)
    
def create_projection_graph(tx):
    query = """
    CALL gds.graph.project.cypher(
      'MbfRux',
      'MATCH (n) 
      WHERE (n:FoodSubType AND 
              EXISTS { MATCH (f:Food)-[:HAS_SUBTYPE]->(n) 
                      WHERE NOT f.group IN ["dishes", "snack_foods", "frozen_desserts", "other_beverages", "confectioneries", "beverages"] }
            ) 
          OR n:Compound 
          OR n:Aroma
      RETURN id(n) AS id, labels(n) AS labels',
      'MATCH (n)-[r:CONTAINS|SCENTED]-(m) 
      RETURN id(n) AS source, id(m) AS target, type(r) AS type',
      { validateRelationships: false }
    )
    """
    
def get_flavor_importance(tx):
    query = """
    MATCH (e:Entry)-[:HAS_FLAVOR]->(f:Flavor)
    RETURN e.name AS entry_name, COUNT(f) AS num_flavors, AVG(f.importance) AS avg_importance
    """
    return list(tx.run(query))

with driver.session() as session:
    data = session.read_transaction(get_flavor_importance)

df = pd.DataFrame(data, columns=["entry_name", "num_flavors", "avg_importance"])
print(df)


/var/folders/g4/fbyxhbrx6h3_4s4m15x3bly00000gn/T/ipykernel_97668/1536840072.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  data = session.read_transaction(get_flavor_importance)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: importance)} {position: line: 3, column: 65, offset: 111} for query: '\n    MATCH (e:Entry)-[:HAS_FLAVOR]->(f:Flavor)\n    RETURN e.name AS entry_name, COUNT(f) AS num_flavors, AVG(f.importance) AS avg_importance\n    '


Empty DataFrame
Columns: [entry_name, num_flavors, avg_importance]
Index: []


In [17]:

X = df.drop(columns=["entry_name"])
y = [1, 0, 1, 1, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier()
model.fit(X_train, y_train)

xgb.plot_importance(model)


ValueError: Found input variables with inconsistent numbers of samples: [0, 5]